# ICA decoding — Part 1 (Feature building)
This notebook mirrors your PCA Part 1, but uses ICA timecourses (`*_ICsK{K}.*`).

**Run this first** to generate:
- `features_static_nonZ_K20.csv`
- `features_ln_conn_pearsonZ_K20.csv`
- `features_delta_conn_pearsonZ_K20.csv`
- `feature_build_manifest_K20.csv`


## 0) Imports + settings

In [1]:

# =========================
# PART 1 — ICA FEATURE BUILDING (UPDATED: saves Ln connectivity too)
# Mirrors your PCA Part 1 exactly, but uses IC naming + file pattern *_ICsK{K}.*
# =========================

from pathlib import Path
import re
import numpy as np
import pandas as pd
from typing import Optional

# ---- USER SETTINGS ----
ROOT = Path(r"/Users/onilarasanjala/Desktop/TSeme/CogNeuSci/CodeData/NewICA")  # <-- CHANGE THIS IF NEEDED
TC_DIR = ROOT / "timecourses"
LABELS_CSV = ROOT / "proficiency_labels.csv"  # must exist in ROOT
TASK_L1 = "compL1"
TASK_LN = "compLn"
K = 20                                   # <-- ICA K (change later for 5,10,15,25,30...)
TR: Optional[float] = None               # set to TR (seconds) or keep None

OUT_STATIC      = ROOT / f"features_static_nonZ_K{K}.csv"
OUT_DELTA_CONN  = ROOT / f"features_delta_conn_pearsonZ_K{K}.csv"
OUT_LN_CONN     = ROOT / f"features_ln_conn_pearsonZ_K{K}.csv"
OUT_MANIFEST    = ROOT / f"feature_build_manifest_K{K}.csv"


# ---- FILE DISCOVERY ----
def discover_timecourses(tc_dir: Path, K: int):
    """
    Finds ICA timecourse files like:
      sub-01_task-compL1_ICsK20.npy  or  .csv
    Returns dict: data[sub][task] = {"npy": path or None, "csv": path or None}
    """
    patt = re.compile(r"(sub-\d+)_task-([A-Za-z0-9]+)_ICsK(\d+)\.(npy|csv)$")
    data = {}
    for p in tc_dir.iterdir():
        m = patt.match(p.name)
        if not m:
            continue
        sub, task, k_str, ext = m.group(1), m.group(2), m.group(3), m.group(4)
        if int(k_str) != K:
            continue
        data.setdefault(sub, {}).setdefault(task, {"npy": None, "csv": None})
        data[sub][task][ext] = p
    return data


def load_tc(entry: dict) -> np.ndarray:
    """
    Prefer npy if present.
    Returns X shape (T, K)
    """
    if entry.get("npy") is not None and entry["npy"].exists():
        X = np.load(entry["npy"]).astype(np.float64)
    elif entry.get("csv") is not None and entry["csv"].exists():
        X = pd.read_csv(entry["csv"]).values.astype(np.float64)
    else:
        raise FileNotFoundError("No npy/csv found for this entry.")

    if X.ndim != 2:
        raise ValueError(f"Bad timecourse shape {X.shape}")
    return X


# ---- FEATURE FUNCTIONS ----
def lag1_autocorr(x: np.ndarray) -> float:
    x0 = x[:-1]
    x1 = x[1:]
    s0 = x0.std()
    s1 = x1.std()
    if s0 == 0 or s1 == 0:
        return 0.0
    return float(np.corrcoef(x0, x1)[0, 1])


def spectral_low_ratio(x: np.ndarray, tr: float, f_low=(0.01, 0.1)) -> float:
    """
    Ratio of power in [0.01,0.1] Hz to total power (excluding DC).
    Requires TR (seconds). If TR is None, we skip this feature.
    """
    x = x - x.mean()
    n = len(x)
    freqs = np.fft.rfftfreq(n, d=tr)
    Xf = np.fft.rfft(x)
    psd = (np.abs(Xf) ** 2)

    # exclude DC (0 Hz)
    valid = freqs > 0
    freqs = freqs[valid]
    psd = psd[valid]
    if psd.sum() == 0:
        return 0.0

    band = (freqs >= f_low[0]) & (freqs <= f_low[1])
    return float(psd[band].sum() / psd.sum())


def per_ic_features(X: np.ndarray, prefix: str, tr: Optional[float]) -> dict:
    """
    X: (T, K)
    Per IC features that survive centering:
      - logstd
      - ac1
      - lowRatio (optional if TR is provided)
    """
    eps = 1e-12
    feats = {}
    for k in range(X.shape[1]):
        ic = f"IC{k+1:02d}"
        col = X[:, k]
        std = col.std(ddof=0)
        feats[f"{prefix}_{ic}_logstd"] = float(np.log(std + eps))
        feats[f"{prefix}_{ic}_ac1"] = lag1_autocorr(col)
        if tr is not None:
            feats[f"{prefix}_{ic}_lowRatio"] = spectral_low_ratio(col, tr=tr)
    return feats


def fisher_z_corr(X: np.ndarray) -> np.ndarray:
    """
    Pearson correlation matrix -> Fisher z transform, clipped.
    """
    C = np.corrcoef(X, rowvar=False)
    np.fill_diagonal(C, 0.0)
    C = np.clip(C, -0.999999, 0.999999)
    Z = np.arctanh(C)
    np.fill_diagonal(Z, 0.0)
    return Z


def upper_triangle_features(M: np.ndarray, prefix: str) -> dict:
    """
    Flatten upper triangle (i<j) into named features.
    """
    K = M.shape[0]
    feats = {}
    for i in range(K):
        for j in range(i + 1, K):
            feats[f"{prefix}_IC{i+1:02d}_IC{j+1:02d}"] = float(M[i, j])
    return feats


# ---- BUILD FEATURE TABLES ----
tc_index = discover_timecourses(TC_DIR, K=K)
subjects = sorted(tc_index.keys())

rows_static = []
rows_conn_delta = []
rows_conn_ln = []          # Ln connectivity
manifest_rows = []
missing_pairs = 0

for sub in subjects:
    if TASK_L1 not in tc_index[sub] or TASK_LN not in tc_index[sub]:
        missing_pairs += 1
        continue

    X1 = load_tc(tc_index[sub][TASK_L1])
    X2 = load_tc(tc_index[sub][TASK_LN])

    if X1.shape[1] != K or X2.shape[1] != K:
        raise ValueError(f"{sub} wrong K: {X1.shape}, {X2.shape}")

    # static features per task
    f1 = per_ic_features(X1, "L1", tr=TR)
    f2 = per_ic_features(X2, "Ln", tr=TR)

    # delta features (Ln - L1) for each feature
    delta = {}
    for key, v in f1.items():
        ln_key = key.replace("L1_", "Ln_")
        delta[key.replace("L1_", "DELTA_")] = f2[ln_key] - v

    row = {"subject": sub}
    row.update(f1)
    row.update(f2)
    row.update(delta)
    rows_static.append(row)

    # connectivity: Ln, L1, and DELTA
    Z1 = fisher_z_corr(X1)
    Z2 = fisher_z_corr(X2)
    ZD = Z2 - Z1

    row_ln = {"subject": sub}
    row_ln.update(upper_triangle_features(Z2, prefix="Ln_zcorr"))
    rows_conn_ln.append(row_ln)

    rowd = {"subject": sub}
    rowd.update(upper_triangle_features(ZD, prefix="DELTA_zcorr"))
    rows_conn_delta.append(rowd)

    manifest_rows.append({
        "subject": sub,
        "T_L1": X1.shape[0],
        "T_Ln": X2.shape[0],
        "has_TR_features": TR is not None
    })

static_df = pd.DataFrame(rows_static).set_index("subject").sort_index()
ln_conn_df = pd.DataFrame(rows_conn_ln).set_index("subject").sort_index()
delta_conn_df = pd.DataFrame(rows_conn_delta).set_index("subject").sort_index()
manifest_df = pd.DataFrame(manifest_rows)

print("Built static feature table:", static_df.shape)
print("Built Ln connectivity table:", ln_conn_df.shape)
print("Built DELTA connectivity table:", delta_conn_df.shape)
print("Missing L1/Ln pairs:", missing_pairs)

static_df.to_csv(OUT_STATIC)
ln_conn_df.to_csv(OUT_LN_CONN)
delta_conn_df.to_csv(OUT_DELTA_CONN)
manifest_df.to_csv(OUT_MANIFEST, index=False)

print("Saved:", OUT_STATIC)
print("Saved:", OUT_LN_CONN)
print("Saved:", OUT_DELTA_CONN)
print("Saved:", OUT_MANIFEST)


Built static feature table: (26, 120)
Built Ln connectivity table: (26, 190)
Built DELTA connectivity table: (26, 190)
Missing L1/Ln pairs: 0
Saved: /Users/onilarasanjala/Desktop/TSeme/CogNeuSci/CodeData/NewICA/features_static_nonZ_K20.csv
Saved: /Users/onilarasanjala/Desktop/TSeme/CogNeuSci/CodeData/NewICA/features_ln_conn_pearsonZ_K20.csv
Saved: /Users/onilarasanjala/Desktop/TSeme/CogNeuSci/CodeData/NewICA/features_delta_conn_pearsonZ_K20.csv
Saved: /Users/onilarasanjala/Desktop/TSeme/CogNeuSci/CodeData/NewICA/feature_build_manifest_K20.csv


# ICA decoding — Part 2 (Modeling + validation)
This mirrors your PCA Part 2, using the ICA feature files created in Part 1.

It prints results for:
- **L1-only**
- **Ln-only**
- **Delta-only (Ln−L1)**
- **Full (L1+Ln+Delta)**


In [2]:

# =========================
# PART 2 — ICA MODELING + VALIDATION
# Mirrors your PCA Part 2 exactly, but uses ICA feature files (IC-based names).
# =========================
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.model_selection import (
    LeaveOneOut,
    RepeatedStratifiedKFold,
    StratifiedKFold,
    cross_val_predict,
    permutation_test_score
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, balanced_accuracy_score

ROOT = Path(r"/Users/onilarasanjala/Desktop/TSeme/CogNeuSci/CodeData/NewICA")  # <-- SAME AS PART 1
LABELS_CSV = ROOT / "proficiency_labels.csv"
K = 20

FEAT_STATIC = ROOT / f"features_static_nonZ_K{K}.csv"
FEAT_LN_CONN = ROOT / f"features_ln_conn_pearsonZ_K{K}.csv"   # used in Part 3
FEAT_DELTA_CONN = ROOT / f"features_delta_conn_pearsonZ_K{K}.csv"

labels = pd.read_csv(LABELS_CSV).set_index("subject")
labels["group"] = labels["group"].str.lower().str.strip()
labels["y"] = (labels["group"] == "advanced").astype(int)

Xstatic = pd.read_csv(FEAT_STATIC).set_index("subject")

df = Xstatic.join(labels[["y"]], how="inner")
y = df["y"].to_numpy()

X_L1    = df.filter(regex=r"^L1_").copy()
X_Ln    = df.filter(regex=r"^Ln_").copy()
X_Delta = df.filter(regex=r"^DELTA_").copy()
X_Full  = df.drop(columns=["y"]).copy()

print("N aligned:", df.shape[0], "class counts:", labels["group"].value_counts().to_dict())
print("Shapes:", "L1", X_L1.shape, "Ln", X_Ln.shape, "Delta", X_Delta.shape, "Full", X_Full.shape)

def fixed_pipeline(C=1.0, penalty="l2"):
    return Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            solver="liblinear",
            max_iter=5000,
            C=C,
            penalty=penalty
        ))
    ])

def loocv_auc_with_bootstrap_ci(Xdf, y, pipe, n_boot=5000, seed=0):
    loo = LeaveOneOut()
    prob = cross_val_predict(pipe, Xdf, y, cv=loo, method="predict_proba")[:, 1]
    auc = roc_auc_score(y, prob)

    rng = np.random.default_rng(seed)
    idx = np.arange(len(y))
    boots = []
    for _ in range(n_boot):
        samp = rng.choice(idx, size=len(idx), replace=True)
        if len(np.unique(y[samp])) < 2:
            continue
        boots.append(roc_auc_score(y[samp], prob[samp]))
    boots = np.array(boots)
    ci_lo, ci_hi = np.quantile(boots, [0.025, 0.975])
    return float(auc), (float(ci_lo), float(ci_hi))

def repeated_cv_auc_dist(Xdf, y, pipe, n_splits=5, n_repeats=200, seed=42):
    rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=seed)
    aucs, bals = [], []
    for tr, te in rskf.split(Xdf, y):
        pipe.fit(Xdf.iloc[tr], y[tr])
        p = pipe.predict_proba(Xdf.iloc[te])[:, 1]
        pred = (p >= 0.5).astype(int)
        aucs.append(roc_auc_score(y[te], p))
        bals.append(balanced_accuracy_score(y[te], pred))
    return np.array(aucs), np.array(bals)

def minimal_nested_tuning(Xdf, y,
                         C_grid=(0.001, 0.01, 0.1, 1, 10, 100, 1000),
                         penalty_grid=("l2", "l1"),
                         outer_splits=5, outer_repeats=100, inner_splits=5, seed=42):
    outer = RepeatedStratifiedKFold(n_splits=outer_splits, n_repeats=outer_repeats, random_state=seed)
    inner = StratifiedKFold(n_splits=inner_splits, shuffle=True, random_state=seed)

    chosen = []
    outer_auc = []

    for tr, te in outer.split(Xdf, y):
        X_tr, X_te = Xdf.iloc[tr], Xdf.iloc[te]
        y_tr, y_te = y[tr], y[te]

        best_auc = -np.inf
        best_hp = None

        for pen in penalty_grid:
            for C in C_grid:
                pipe = fixed_pipeline(C=C, penalty=pen)
                inner_aucs = []
                for tr2, te2 in inner.split(X_tr, y_tr):
                    pipe.fit(X_tr.iloc[tr2], y_tr[tr2])
                    p = pipe.predict_proba(X_tr.iloc[te2])[:, 1]
                    inner_aucs.append(roc_auc_score(y_tr[te2], p))
                m = float(np.mean(inner_aucs))
                if m > best_auc:
                    best_auc, best_hp = m, (pen, C)

        pen, C = best_hp
        pipe = fixed_pipeline(C=C, penalty=pen)
        pipe.fit(X_tr, y_tr)
        p = pipe.predict_proba(X_te)[:, 1]
        outer_auc.append(float(roc_auc_score(y_te, p)))
        chosen.append(best_hp)

    from collections import Counter
    counts = Counter(chosen)
    return float(np.mean(outer_auc)), float(np.std(outer_auc)), counts

def fast_permutation_pvalue(Xdf, y, pipe, n_perm=2000, seed=0):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    score, perm_scores, pvalue = permutation_test_score(
        pipe, Xdf, y,
        scoring="roc_auc",
        cv=cv,
        n_permutations=n_perm,
        n_jobs=-1,
        random_state=seed
    )
    return float(score), float(pvalue)

def run_bundle(name, Xdf):
    print("\n======", name, "======")
    pipe = fixed_pipeline(C=1.0, penalty="l2")

    auc_loocv, ci = loocv_auc_with_bootstrap_ci(Xdf, y, pipe, n_boot=5000, seed=0)
    print("LOOCV AUC:", auc_loocv, "Bootstrap 95% CI:", ci)

    aucs, bals = repeated_cv_auc_dist(Xdf, y, pipe, n_splits=5, n_repeats=200, seed=42)
    print("RepCV AUC mean±sd:", float(aucs.mean()), float(aucs.std()))
    print("RepCV BAL mean±sd:", float(bals.mean()), float(bals.std()))

    outer_mean, outer_sd, counts = minimal_nested_tuning(Xdf, y, outer_repeats=50)
    print("Nested (tune C/penalty only) outer AUC mean±sd:", outer_mean, outer_sd)
    print("Top hyperparams:", counts.most_common(5))

    score, pval = fast_permutation_pvalue(Xdf, y, pipe, n_perm=2000, seed=0)
    print("Permutation (5-fold) AUC score:", score, "p-value:", pval)

# Run models
run_bundle("L1-only (negative control)", X_L1)
run_bundle("Ln-only", X_Ln)
run_bundle("Delta-only", X_Delta)
run_bundle("FULL (L1+Ln+Delta)", X_Full)


N aligned: 26 class counts: {'advanced': 15, 'intermediate': 11}
Shapes: L1 (26, 40) Ln (26, 40) Delta (26, 40) Full (26, 120)

====== L1-only (negative control) ======
LOOCV AUC: 0.6424242424242423 Bootstrap 95% CI: (0.4106894841269841, 0.8472222222222222)
RepCV AUC mean±sd: 0.6247222222222223 0.25350003652567493
RepCV BAL mean±sd: 0.5929166666666666 0.20992351451051033
Nested (tune C/penalty only) outer AUC mean±sd: 0.5371111111111111 0.2476916641589441
Top hyperparams: [(('l1', 1), 47), (('l2', 0.001), 33), (('l1', 10), 30), (('l2', 1), 29), (('l1', 1000), 25)]
Permutation (5-fold) AUC score: 0.6444444444444443 p-value: 0.25487256371814093

====== Ln-only ======
LOOCV AUC: 0.6181818181818182 Bootstrap 95% CI: (0.38181818181818183, 0.8273809523809524)
RepCV AUC mean±sd: 0.6105 0.25390091753659716
RepCV BAL mean±sd: 0.5514166666666667 0.20639617715559666
Nested (tune C/penalty only) outer AUC mean±sd: 0.5668888888888889 0.24642119918425728
Top hyperparams: [(('l2', 0.001), 132), (('l2

# ICA decoding — Part 3 (Ln-only + top-IC Ln connectivity)
This mirrors your PCA Part 3 exactly:
1) Train Ln-only model on TRAIN folds
2) Pick **top 5 ICs** from TRAIN-only weights
3) Add Ln connectivity edges among those 5 ICs
4) Evaluate with repeated CV + **manual permutation test** (leakage-safe)


In [5]:

# =========================
# PART 3 — LN-ONLY + TOP-IC LN CONNECTIVITY (LEAKAGE-SAFE)
# + Manual permutation test (publication-friendly; avoids sklearn tag issues)
# Mirrors your PCA Part 3, but uses IC naming.
# =========================

from pathlib import Path
import numpy as np
import pandas as pd
import re
from collections import Counter
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


# -------------------------
# Basic model builder
# -------------------------
def fixed_pipe(C=1.0, penalty="l2"):
    return Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            solver="liblinear",
            max_iter=5000,
            C=C,
            penalty=penalty
        ))
    ])


# -------------------------
# Utilities
# -------------------------
def ic_group_importance_from_coef(feature_names, coef):
    """Sum |coef| by IC index extracted from names like Ln_IC07_..."""
    ic_re = re.compile(r"IC(\d{2})")
    scores = {}
    for name, w in zip(feature_names, coef):
        m = ic_re.search(name)
        if not m:
            continue
        ic = int(m.group(1))
        scores[ic] = scores.get(ic, 0.0) + abs(float(w))
    return scores

def connectivity_cols_for_ics(ics, prefix="Ln_zcorr"):
    ics = sorted(ics)
    cols = []
    for i in range(len(ics)):
        for j in range(i + 1, len(ics)):
            a, b = ics[i], ics[j]
            cols.append(f"{prefix}_IC{a:02d}_IC{b:02d}")
    return cols

def sanity_check_inputs(Xbase: pd.DataFrame, conn: pd.DataFrame, y: np.ndarray, conn_prefix: str):
    assert isinstance(Xbase, pd.DataFrame) and isinstance(conn, pd.DataFrame)
    assert len(Xbase) == len(y), "Xbase and y length mismatch"
    assert Xbase.index.equals(conn.index), "Xbase and conn indices must match exactly"
    assert set(np.unique(y)).issubset({0, 1}), "y must be binary 0/1"

    pref_cols = [c for c in conn.columns if c.startswith(conn_prefix + "_IC")]
    if len(pref_cols) == 0:
        raise ValueError(f"No connectivity columns found with prefix='{conn_prefix}'.")
    print(f"[OK] Found {len(pref_cols)} connectivity columns with prefix '{conn_prefix}'.")


# -------------------------
# Core: single CV run (supports either real y or permuted y)
# -------------------------
def cv_auc_foldwise_augmented(
    Xbase: pd.DataFrame,
    conn: pd.DataFrame,
    y: np.ndarray,
    splits,
    conn_prefix: str = "Ln_zcorr",
    top_m: int = 5,
    C_base: float = 1.0,
    C_final: float = 1.0,
    penalty: str = "l2",
    return_debug: bool = False
):
    aucs = []
    picked_sets = []
    edge_use_counter = Counter()
    missing_edges_total = 0

    for tr, te in splits:
        Xtr, Xte = Xbase.iloc[tr], Xbase.iloc[te]
        ytr, yte = y[tr], y[te]

        # 1) pick top ICs from TRAIN only (Ln-only)
        base_model = fixed_pipe(C=C_base, penalty=penalty)
        base_model.fit(Xtr, ytr)
        w = base_model.named_steps["clf"].coef_.ravel()
        scores = ic_group_importance_from_coef(Xtr.columns, w)
        top = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)[:top_m]
        top_ics = [ic for ic, _ in top]
        picked_sets.append(tuple(top_ics))

        # 2) add connectivity edges among those ICs
        wanted_cols = connectivity_cols_for_ics(top_ics, prefix=conn_prefix)
        cols = [c for c in wanted_cols if c in conn.columns]
        missing_edges_total += (len(wanted_cols) - len(cols))
        edge_use_counter.update(cols)

        Xtr_aug = pd.concat([Xtr, conn.iloc[tr][cols]], axis=1)
        Xte_aug = pd.concat([Xte, conn.iloc[te][cols]], axis=1)

        # 3) fit final model and score AUC
        final_model = fixed_pipe(C=C_final, penalty=penalty)
        final_model.fit(Xtr_aug, ytr)
        p = final_model.predict_proba(Xte_aug)[:, 1]

        # If test fold has only one class, AUC undefined -> skip fold
        if len(np.unique(yte)) < 2:
            continue

        aucs.append(roc_auc_score(yte, p))

    auc_mean = float(np.mean(aucs))

    if not return_debug:
        return auc_mean

    n_folds = len(aucs)
    return {
        "auc_mean": auc_mean,
        "auc_sd": float(np.std(aucs)),
        "n_folds": int(n_folds),
        "most_common_ic_sets": Counter(picked_sets).most_common(10),
        "most_used_edges": edge_use_counter.most_common(15),
        "avg_missing_edges_per_fold": float(missing_edges_total / n_folds) if n_folds > 0 else float("nan")
    }


# -------------------------
# Repeated CV summary
# -------------------------
def repeated_cv_summary_augmented(
    Xbase: pd.DataFrame,
    conn: pd.DataFrame,
    y: np.ndarray,
    conn_prefix: str = "Ln_zcorr",
    top_m: int = 5,
    C_base: float = 1.0,
    C_final: float = 1.0,
    penalty: str = "l2",
    n_splits: int = 5,
    n_repeats: int = 200,
    seed: int = 42
):
    sanity_check_inputs(Xbase, conn, y, conn_prefix=conn_prefix)
    expected_edges = top_m * (top_m - 1) // 2
    print(f"[INFO] top_m={top_m} -> expected edges per fold = C({top_m},2) = {expected_edges}")

    rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=seed)
    splits = list(rskf.split(Xbase, y))

    out = cv_auc_foldwise_augmented(
        Xbase, conn, y, splits,
        conn_prefix=conn_prefix,
        top_m=top_m,
        C_base=C_base,
        C_final=C_final,
        penalty=penalty,
        return_debug=True
    )

    print(f"[INFO] Avg missing edges per fold: {out['avg_missing_edges_per_fold']:.3f} (should be ~0)")
    out["top_m"] = int(top_m)
    out["expected_edges_per_fold"] = int(expected_edges)
    return out


# -------------------------
# Manual permutation test (publication-friendly)
# -------------------------
def permutation_test_augmented_auc(
    Xbase: pd.DataFrame,
    conn: pd.DataFrame,
    y: np.ndarray,
    conn_prefix: str = "Ln_zcorr",
    top_m: int = 5,
    C_base: float = 1.0,
    C_final: float = 1.0,
    penalty: str = "l2",
    n_splits: int = 5,
    n_perm: int = 2000,
    seed: int = 0,
    verbose_every: int = 200
):
    """
    Returns:
      observed_auc, p_value, perm_aucs (array)
    p-value computed as (1 + #perm >= obs) / (1 + n_perm)
    """
    sanity_check_inputs(Xbase, conn, y, conn_prefix=conn_prefix)

    # Fix CV splits once (important!)
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    splits = list(cv.split(Xbase, y))

    # Observed score
    obs = cv_auc_foldwise_augmented(
        Xbase, conn, y, splits,
        conn_prefix=conn_prefix,
        top_m=top_m,
        C_base=C_base,
        C_final=C_final,
        penalty=penalty,
        return_debug=False
    )

    rng = np.random.default_rng(seed)
    perm_scores = np.empty(n_perm, dtype=float)

    for i in range(n_perm):
        y_perm = rng.permutation(y)  # preserves class counts automatically
        perm_scores[i] = cv_auc_foldwise_augmented(
            Xbase, conn, y_perm, splits,
            conn_prefix=conn_prefix,
            top_m=top_m,
            C_base=C_base,
            C_final=C_final,
            penalty=penalty,
            return_debug=False
        )
        if verbose_every and (i + 1) % verbose_every == 0:
            print(f"[perm] {i+1}/{n_perm} done...")

    p = (1.0 + np.sum(perm_scores >= obs)) / (1.0 + n_perm)
    return float(obs), float(p), perm_scores


# -------------------------
# RUN (Ln-only + Ln connectivity)
# -------------------------
ROOT = Path(r"/Users/onilarasanjala/Desktop/TSeme/CogNeuSci/CodeData/NewICA")
K = 20

LABELS_CSV = ROOT / "proficiency_labels.csv"
FEAT_STATIC = ROOT / f"features_static_nonZ_K{K}.csv"
FEAT_LN_CONN = ROOT / f"features_ln_conn_pearsonZ_K{K}.csv"

labels = pd.read_csv(LABELS_CSV).set_index("subject")
labels["group"] = labels["group"].str.lower().str.strip()
labels["y"] = (labels["group"] == "advanced").astype(int)

Xstatic = pd.read_csv(FEAT_STATIC).set_index("subject")
connLn = pd.read_csv(FEAT_LN_CONN).set_index("subject")

df = Xstatic.join(labels[["y"]], how="inner")
X_Ln = df.filter(regex=r"^Ln_").copy()

# Align (exact index match)
common = X_Ln.index.intersection(connLn.index)
X_base = X_Ln.loc[common].copy()
y_base = labels.loc[common, "y"].to_numpy()
connLn = connLn.loc[common].copy()
connLn = connLn.loc[X_base.index]

print("\n=== Repeated CV summary (sanity-checked): Ln-only base + top-IC Ln connectivity ===")
aug_ln = repeated_cv_summary_augmented(
    Xbase=X_base,
    conn=connLn,
    y=y_base,
    conn_prefix="Ln_zcorr",
    top_m=7,
    C_base=1.0,
    C_final=1.0,
    penalty="l2",
    n_splits=5,
    n_repeats=200,
    seed=42
)
print(aug_ln)

print("\n=== Manual permutation test (leakage-safe) for augmented Ln model ===")
obs_auc, pval, perm_aucs = permutation_test_augmented_auc(
    Xbase=X_base,
    conn=connLn,
    y=y_base,
    conn_prefix="Ln_zcorr",
    top_m=7,
    C_base=1.0,
    C_final=1.0,
    penalty="l2",
    n_splits=5,
    n_perm=2000,     # start 500 if you want quick; 2000 for final
    seed=0,
    verbose_every=200
)
print("Observed 5-fold AUC:", obs_auc)
print("Permutation p-value:", pval)



=== Repeated CV summary (sanity-checked): Ln-only base + top-IC Ln connectivity ===
[OK] Found 190 connectivity columns with prefix 'Ln_zcorr'.
[INFO] top_m=7 -> expected edges per fold = C(7,2) = 21
[INFO] Avg missing edges per fold: 0.000 (should be ~0)
{'auc_mean': 0.7136666666666668, 'auc_sd': 0.24949507033898843, 'n_folds': 1000, 'most_common_ic_sets': [((7, 19, 12, 9, 15, 8, 1), 2), ((1, 5, 19, 12, 6, 7, 10), 2), ((2, 1, 8, 12, 19, 18, 16), 2), ((6, 7, 20, 16, 18, 8, 13), 2), ((7, 19, 9, 15, 20, 8, 5), 2), ((19, 15, 5, 6, 18, 14, 16), 2), ((19, 9, 1, 12, 2, 16, 14), 2), ((1, 19, 2, 15, 9, 12, 18), 2), ((7, 6, 16, 8, 20, 18, 13), 2), ((7, 19, 18, 5, 17, 6, 8), 2)], 'most_used_edges': [('Ln_zcorr_IC06_IC19', 673), ('Ln_zcorr_IC07_IC19', 559), ('Ln_zcorr_IC18_IC19', 551), ('Ln_zcorr_IC15_IC19', 540), ('Ln_zcorr_IC12_IC19', 535), ('Ln_zcorr_IC06_IC07', 486), ('Ln_zcorr_IC06_IC18', 463), ('Ln_zcorr_IC05_IC19', 433), ('Ln_zcorr_IC06_IC15', 413), ('Ln_zcorr_IC09_IC19', 391), ('Ln_zcorr